# Cross-Validation & Regularization

In [ ]:
from sklearn.preprocessing import StandardScaler, PolynomialFeatures, OneHotEncoder
from sklearn.linear_model import Ridge, Lasso, ElasticNet, LinearRegression,\
LassoCV, RidgeCV, ElasticNetCV
from sklearn.model_selection import train_test_split, KFold,\
cross_val_score, cross_validate, ShuffleSplit
import pandas as pd
from sklearn.metrics import mean_squared_error
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

One of the goals of a machine learning project is to make models which are highly predictive.
If the model fails to generalize to unseen data then the model is bad.

## Agenda

SWBAT:

- explain the notion of "validation data";
- explain and use the algorithm of cross-validation;
- explain the concept of regularization;
- use Lasso and Ridge regularization in model design.

## Review of Earlier concepts

### Overfitting vs Underfitting
    1. Underfit models fail to capture all of the information in the data
        1. Ex. Just predicting the mean leaves a lot of information on the table
    2. Overfit models fit to the noise in the data and fail to generalize
    3. How would we know if our model is over- or underfit?
        1. Train test split
        2. Look at the testing error
        3. As model complexity increases so does the possibility for overfitting

### Bias-Variance Tradeoff
    1. High bias
        1. Systematic error in predictions
        2. Bias is about the strength of assumptions the model makes
        3. Underfit models tend to have high bias
    2. High variance
        1. The model is highly sensitive to changes in the data
        2. Overfit models tend to have low bias

#### Example

High bias is easy to wrap one's mind around: Imagine pulling three red balls from an urn that has hundreds of balls of all colors in a uniform distribution. Then my sample is a terrible representative of the whole population. If I were to build a model by extrapolating from my sample, that model would predict that _every_ ball produced would be red! That is, this model would be incredibly biased.

High variance is a little bit harder to visualize, but it's basically the "opposite" of this. Imagine that the population of balls in the urn is mostly red, but also that there are a few balls of other colors floating around. Now imagine that our sample comprises a few balls, none of which is red. In this case, we've essentially picked up on the "noise", rather than the "signal". If I were to build a model by extrapolating from my sample, that model would be needlessly complex. It might predict that balls drawn before noon will be orange and that balls drawn after 8pm will be green, when the reality is that a simple model that predicted 'red' for all balls would be a superior model!

The important idea here is that there is a *trade-off*: If we have too few data in our sample (training set), or too few predictors, we run the risk of high *bias*, i.e. an underfit model. On the other hand, if we have too many predictors (especially ones that are collinear), we run the risk of high *variance*, i.e. an overfit model.

[Here](https://en.wikipedia.org/wiki/Overfitting#/media/File:Overfitting.svg) is a nice illustration of the difficulty.

## Validation

So: You've split your data in two, reserving one piece for training your model and the other for testing it after it's built. That's good!

But generally speaking we want to take more precautions than this. After all, we're still imagining building just one model on the training set and then crossing our fingers for its performance on the test set.

Data scientists often distinguish *three* subsets of data: training, **validation**, and testing.

Roughly:
- Training data is for building the model;
- Validation data is for *tweaking* the model;
- Testing data is for evaluating the model on unseen data.

This "tweaking" includes most of all the fine-tuning of model parameters (see below). Think of what this three-way distinction allows us to do:

I can build a model on some data. Then, **before** I introduce the model to the testing data, I can introduce it to a different batch of data (the validation set). With respect to the validation data I can do things like measure error and tweak model parameters to minimize that error. Of course, I also don't want to lose sight of the error on the training data. If the model error has been minimized on the training error, then of course any changes I make to the model parameters will take me away from that minimum. But still the new information I've gained by looking at the model's performance on the validiation data is valuable. I might for example go with a kind of compromising model whose parameters produce an error that's not too big on the training data and not too big on the validation data.

**Question**: What's different about this procedure from what we've described before? Aren't I just calling the test data "validation data" now? Is there any substantive difference?

### From Validation to Cross-Validation

Since my model will "see" the validation data in any case, I might as well use *all* of my training data to validate my model! How do I do this?

Cross-validation works like this: First I'll partition my training data into $k$-many *folds*. Then I'll train a model on $k-1$ of those folds and "test" it on the remaining fold. I'll do this for all possible divisions of my $k$ folds into $k-1$ training folds and a single "testing" fold. Since there are $k\choose 1$$=k$-many ways of doing this, I'll be building $k$-many models!

#### In Python

In [ ]:
birds = sns.load_dataset('penguins')
birds.sample(5)

In [ ]:
birds.info()

In [ ]:
# For simplicity's sake we'll limit our analysis to the numeric columns.

numeric = birds[['culmen_length_mm', 'culmen_depth_mm',
                 'flipper_length_mm', 'body_mass_g']]

In [ ]:
# We'll drop the rows with null values

numeric = numeric.dropna().reset_index()

Initiating the `KFold()` object

In [ ]:
folds = KFold(n_splits=10, shuffle=True, random_state=42)

In [ ]:
folds.split(numeric)

In [ ]:
[bird for bird in folds.split(numeric)]

Suppose I want to model `body_mass_g` as a function of the other attributes.

In [ ]:
X = numeric.drop('body_mass_g', axis=1)
y = numeric['body_mass_g']

We'll make ten models and record our evaluations of them.

In [ ]:
r2_scores = np.array([])

# For each partition in our list:
for train, test in folds.split(numeric):
    
    # We'll train a model on the nine-fold part ...
    X_training = X.loc[train, :]
    
    # and test it on the fold left over.
    X_testing = X.loc[test, :]
    y_training = y.loc[train]
    y_testing = y.loc[test]
    lr = LinearRegression()
    lr.fit(X_training, y_training)
    
    # We'll keep track of all of the R^2 scores.
    r2_score = lr.score(X_testing, y_testing)
    r2_scores = np.append(r2_scores, r2_score)

In [ ]:
r2_scores

In [ ]:
r2_scores.mean()

In [ ]:
lr2 = LinearRegression()

In [ ]:
cv = ShuffleSplit(random_state=42)

In [ ]:
cross_validate(estimator=lr2, X=X, y=y, cv=cv) # The 'cv' parameter can
                                               # also take an integer value

In [ ]:
cross_validate(estimator=lr2, X=X, y=y, cv=cv)['test_score']

In [ ]:
cross_validate(estimator=lr2, X=X, y=y, cv=cv)['test_score'].mean()

## Regularization - a way to prevent overfitting
    - Types of regularization
        1. Reducing the number of features
        2. Increasing the amount of data
        3. Ridge, Lasso, Elastic Net
        
Again, complex models are very flexible in the patterns that they can model but this also means that they can easily find patterns that are simply statistical flukes of one particular dataset rather than patterns reflective of the underlying data-generating process.

### The Strategy Behind Ridge / Lasso / Elastic Net

Overfit models overestimate the relevance that predictors have for a target. Thus overfit models tend to have **overly large coefficients**.

The evaluation of many models, linear regression included, proceeds by measuring its **error**, some quantifiable expression of the discrepancy between its predictions and the ground truth. The best-fit line of LR, for example, minimizes the sum of squared residuals.

Our new idea, then, will be ***to add a term representing the size of our coefficients to our loss function***.

The goal will still be to minimize this new function, but we can make progress toward this minimum *either* by reducing the size of our residuals *or* by reudcing the size of our coefficients.

Since coefficients can be either negative or positive, we have the familiar difficulty that we can't simply add them up to get a sense of how large they are in general. Once again there are two natural choices: We could focus either on the squares or the absolute values of the coefficients. The former strategy is the basis for **Ridge** (also called Tikhonov) regularization; the latter strategy results in **LASSO** (Least Absolute Shrinkage and Selection Operator) regularization.

These tools, as we shall see, are easily implemented with `sklearn`.

#### Changing Our Loss Function

Overfitting is generally a result of high model variance. High model variance can be caused by:
- having irrelevant or too many predictors
- multicollinearity
- large coefficients

The first problem is about picking up on noise rather than signal.
The second problem is about having a least-squares estimate that is highly sensitive to random error.
The third is about having highly sensitive predictors.

Regularization is about introducing a factor into our model designed to enforce the stricture that the coefficients stay small, by penalizing the ones that get too large.

That is, we'll alter our loss function so that the goal now is not merely to minimize the difference between actual values and our model's predicted values. Rather, we'll add in a term to our loss function that represents the sizes of the coefficients.

There are two popular ways of doing this:

Lasso ("L1"): Minimize $\large\Sigma^{n_{obs.}}_{i=1}[(y_i - \Sigma^{n_{feat.}}_{j=0}\beta_j\times x_{ij})^2 + \lambda\Sigma^{n_{feat.}}_{j=0}|\beta_j|]$
<br/> <br/>

Ridge ("L2"): Minimize $\large\Sigma^{n_{obs.}}_{i=1}[(y_i - \Sigma^{n_{feat.}}_{j=0}\beta_j\times x_{ij})^2 + \lambda\Sigma^{n_{feat.}}_{j=0}\beta^2_j]$

**$\rightarrow$ Don't let these formulas be intimidating.** The first term in each of these (the sum of squares) is the same, and is just the familiar loss function that we've always used. What distinguishes the Lasso Regression from the Ridge Regression is only the extra term on the right. The Lasso uses the absolute values of the coefficients, while the Ridge uses the squares of the coefficients.

**When should I use each of these?**

- For a given value of $\lambda$, the ridge makes for a gentler reining in of runaway coefficients. When in doubt, try ridge first.
- The lasso will more quickly reduce the contribution of individual predictors down to insignificance. It is therefore most useful for trimming through the fat of datasets with many predictors or if a model with very few predictors is especially desirable.

For a nice discussion of these methods in Python, see [this post](https://towardsdatascience.com/ridge-and-lasso-regression-a-complete-guide-with-python-scikit-learn-e20e34bcbf0b).

## Ridge and Lasso Regression

### Producing an Overfit Model

We can often produce an overfit model by including **interaction terms**. We'll start over with the penguins dataset. This time we'll include the categorical features.

#### Train-Test Split

In [ ]:
birds = birds.dropna()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(birds.drop('body_mass_g',
                                                              axis=1),
                                                   birds['body_mass_g'],
                                                   random_state=42)

In [ ]:
ohe = OneHotEncoder(drop='first')
dummies = ohe.fit_transform(X_train[['species', 'island', 'sex']])
dummies_df = pd.DataFrame(dummies.todense(), columns=ohe.get_feature_names(),
                         index=X_train.index)
X_train_df = pd.concat([X_train[['culmen_length_mm', 'culmen_depth_mm',
                                'flipper_length_mm']], dummies_df], axis=1)
X_train_df.head()

#### First simple model

In [ ]:
test_dummies = ohe.transform(X_test[['species', 'island', 'sex']])
test_df = pd.DataFrame(test_dummies.todense(), columns=ohe.get_feature_names(),
                       index=X_test.index)
X_test_df = pd.concat([X_test[['culmen_length_mm', 'culmen_depth_mm',
                              'flipper_length_mm']], test_df], axis=1)

In [ ]:
lr1 = LinearRegression()
lr1.fit(X_train_df, y_train)

pens_preds = lr1.predict(X_test_df)

In [ ]:
lr1.score(X_train_df, y_train)

In [ ]:
lr1.score(X_test_df, y_test)

In [ ]:
np.sqrt(mean_squared_error(pens_preds, y_test))

#### Add Polynomial Features

In [ ]:
pf = PolynomialFeatures(degree=3)
X_poly_train = pf.fit_transform(X_train_df)

In [ ]:
poly_lr = LinearRegression()
poly_lr.fit(X_poly_train, y_train)

In [ ]:
X_poly_test = pf.transform(X_test_df)
poly_preds = poly_lr.predict(X_poly_test)

In [ ]:
poly_lr.score(X_poly_train, y_train)

That's an improvement in $R^2$ on the training data. Let's check the score on the test data.

In [ ]:
poly_lr.score(X_poly_test, y_test)

Ugh! What happened?

In [ ]:
np.sqrt(mean_squared_error(poly_preds, y_test))

In a word, we've overfit our model. Let's see if ridge regularization can help.

### Ridge (L2) Regression

In [ ]:
ss = StandardScaler()
pf = PolynomialFeatures(degree=3)

# You should always be sure to _standardize_ your data before
# applying regularization!

X_train_processed = pf.fit_transform(ss.fit_transform(X_train_df))
X_test_processed = pf.transform(ss.transform(X_test_df))

In [ ]:
# 'Lambda' is the standard variable for the strength of the
# regularization (as in the above formulas), but since lambda
# is a key word in Python, these sklearn regularization tools
# use 'alpha' instead.

rr = Ridge(alpha=10, random_state=42)

rr.fit(X_train_processed, y_train)
ridge_preds = rr.predict(X_test_processed)

In [ ]:
rr.score(X_train_processed, y_train)

In [ ]:
rr.score(X_test_processed, y_test)

In [ ]:
np.sqrt(mean_squared_error(ridge_preds, y_test))

Much better! But how do we know which value of `alpha` to pick?

### Crossvalidation to Optimize the Regularization Hyperparameter

The regularization strength could sensibly be any nonnegative number, so there's no way to check "all possible" values. It's often useful to try several values that are different orders of magnitude.

In [ ]:
alphas = [1, 10, 100, 1000, 10000]
train_scores = []
test_scores = []

for alpha in alphas:
    rr = Ridge(alpha=alpha, random_state=42)
    rr.fit(X_train_processed, y_train)
    train_score = rr.score(X_train_processed, y_train)
    test_score = rr.score(X_test_processed, y_test)
    
    train_scores.append(train_score)
    test_scores.append(test_score)

In [ ]:
plt.style.use('fivethirtyeight')
fig, ax = plt.subplots()
plt.xscale('log')
plt.title('Ridge $R^2$ as a function of regularization strength')
ax.set_xlabel('Regularization strength $\lambda$')
ax.set_ylabel('$R^2$')
ax.plot(alphas, train_scores, label='train')
ax.plot(alphas, test_scores, label='test')
plt.legend();

It looks like the best value is somewhere around 100. If we wanted more precision, we could repeat the same sort of exercise with a set of alphas nearer to 100.

#### Observation
Notice how the values increase but then decrease? Regularization helps with overfitting, but if the strength of the regularization becomes too great, then large coefficients will be punished more than they really should. What happens then is that the original error between truth and model predictions becomes neglected as a quantity to be minimized, and the bias of the model begins to outweigh its variance.

### Lasso (L1) Regression

**Exercise**: Produce a similar plot using `Lasso` instead of `Ridge`!
    
    Hint: You may need to increase the value of the 'max_iter' parameter over the default.
    Level Up: Record the coefficients of each model to see how many go to 0 for each value of alpha.

In [ ]:
alphas = [1, 10, 100, 1000, 10000]
train_scores = []
test_scores = []
coefs = []

In [ ]:
for alpha in alphas:
    lr = Lasso(alpha=alpha, random_state=42, max_iter=100000)
    lr.fit(X_train_processed, y_train)
    coefs.append(lr.coef_)
    train_score = lr.score(X_train_processed, y_train)
    test_score = cross_val_score(lr, X_test_processed, y_test).mean()
    
    train_scores.append(train_score)
    test_scores.append(test_score)

In [ ]:
fig, ax = plt.subplots()
plt.xscale('log')
plt.title('Lasso $R^2$ as a function of regularization strength')
ax.set_xlabel('Regularization strength $\lambda$')
ax.set_ylabel('$R^2$')
ax.plot(alphas, train_scores, label='train')
ax.plot(alphas, test_scores, label='test')
plt.legend();

In [ ]:
[len(coef[coef!=0]) for coef in coefs]

### Elastic Net

There is a combination of L1 and L2 regularization called the Elastic Net that can also be used. The idea is to use a scaled linear combination of the lasso and the ridge, where the weights add up to 100%. We might want 50% of each, but we also might want, say, 10% Lasso and 90% Ridge.

The loss function for an Elastic Net Regression looks like this:

Elastic Net: Minimize

$\rho\Sigma^{n_{obs.}}_{i=1}[(y_i - \Sigma^{n_{feat.}}_{j=0}\beta_j\times x_{ij})^2 + \lambda\Sigma^{n_{feat.}}_{j=0}|\beta_j|] + (1 - \rho)\Sigma^{n_{obs.}}_{i=1}[(y_i - \Sigma^{n_{feat.}}_{j=0}\beta_j\times x_{ij})^2 + \lambda\Sigma^{n_{feat.}}_{j=0}\beta^2_j]$

Sometimes you will see this loss function represented with different scaling terms, but the basic idea is to have a combination of L1 and L2 regularization terms.

#### Coding the Elastic Net

Naturally, the Elastic Net has the same interface through sklearn as the other regularization tools! The only difference is that we now have to specify how much of each regularization term we want. The name of the parameter for this (represented by $\rho$ above) in sklearn is `l1_ratio`.

In [ ]:
enet = ElasticNet(alpha=10, l1_ratio=0.1, random_state=42)

enet.fit(X_train_processed, y_train)

In [ ]:
enet.score(X_train_processed, y_train)

In [ ]:
enet.score(X_test_processed, y_test)

Setting the `l1_ratio` to 1 is equivalent to the lasso:

In [ ]:
ratios = np.linspace(0.01, 1, 100)

In [ ]:
preds = []
for ratio in ratios:
    enet = ElasticNet(alpha=100, l1_ratio=ratio, random_state=42)
    enet.fit(X_train_processed, y_train)
    preds.append(enet.predict(X_test_processed[0].reshape(1, -1)))

In [ ]:
fig, ax = plt.subplots()

lasso = Lasso(alpha=100, random_state=42)
lasso.fit(X_train_processed, y_train)
lasso_pred = lasso.predict(X_test_processed[0].reshape(1, -1))

ax.plot(ratios, preds, label='elastic net')
ax.scatter(1, lasso_pred, c='k', s=70, label='lasso')
plt.legend();

#### Note on `ElasticNet()`

Is an Elastic Net with `l1_ratio` set to 0 equivalent to the ridge? In theory yes. But in practice no. It looks like the `ElasticNet()` predictions on the first test data point as `l1_ratio` shrinks are tending toward some value around 3400. Let's check to see what prediction `Ridge()` gives us:

In [ ]:
ridge = Ridge(alpha=10, random_state=42)
ridge.fit(X_train_processed, y_train)
ridge.predict(X_test_processed[0].reshape(1, -1))[0]

If you check the docstring for the `ElasticNet()` class you will see:
- that the function being minimized is slightly different from what we saw above; and
- that the results are unreliable when `l1_ratio` $\leq 0.01$.

**Exercise**: Visualize the difference in this case between `ElasticNet(l1_ratio=0.01)` and `Ridge()` by making a scatterplot of each model's predicted values for the first ten points in `X_test_processed`. Use `alpha=10` for each model.

        Level Up: Make a second scatterplot that compares the predictions on the same data
        points between ElasticNet(l1_ratio=1) and Lasso().

In [ ]:
fig, ax = plt.subplots()

enet_r = ElasticNet(alpha=10, l1_ratio=0.01, random_state=42)
enet_r.fit(X_train_processed, y_train)
preds_enr = enet_r.predict(X_test_processed[:10])

preds_ridge = ridge.predict(X_test_processed[:10])

ax.scatter(np.arange(10), preds_enr)
ax.scatter(np.arange(10), preds_ridge);

In [ ]:
# Level Up

fig, ax = plt.subplots()

enet_l = ElasticNet(alpha=10, l1_ratio=1, random_state=42)
enet_l.fit(X_train_processed, y_train)
preds_enl = enet_l.predict(X_test_processed[:10])

preds_lasso = lasso.predict(X_test_processed[:10])

ax.scatter(np.arange(10), preds_enl)
ax.scatter(np.arange(10), preds_lasso);

### Fitting Regularized Models with Cross-Validation

Our friend `sklearn` also includes tools that fit regularized regressions *with cross-validation*: `LassoCV`, `RidgeCV`, and `ElasticNetCV`.

**Exercise**: Use `RidgeCV` to fit a seven-fold cross-validated ridge regression model to our `X_train_processed` data and then calculate $R^2$ and the RMSE (root-mean-squared error) on our test set.

In [ ]:
rcv = RidgeCV(cv=7)
rcv.fit(X_train_processed, y_train)

In [ ]:
rcv.score(X_test_processed, y_test)

In [ ]:
np.sqrt(mean_squared_error(y_test, rcv.predict(X_test_processed)))